In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
metadata = pd.read_csv('movies_metadata.csv',low_memory=False)

In [4]:
tfidf = TfidfVectorizer(stop_words='english',min_df=2,max_df=0.7)

metadata['overview'] = metadata['overview'].fillna('')

In [5]:
 tfidf_matrix = tfidf.fit_transform(metadata['overview'])

In [6]:
tfidf_matrix.shape

(45466, 37664)

In [7]:
tfidf.get_feature_names()[5000:5010]
cosine_sim = cosine_similarity(tfidf_matrix)

In [8]:
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

MemoryError: Unable to allocate 15.4 GiB for an array with shape (45466, 45466) and data type float64

In [66]:
cosine_sim.shape

(1000, 1000)

In [67]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [73]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]


In [74]:
get_recommendations('Toy Story')

485                                          Malice
448                               For Love or Money
748                              I Shot Andy Warhol
591                                 Window to Paris
314                        The Shawshank Redemption
180                                    Mute Witness
928                                Bringing Up Baby
370                                    Safe Passage
427    City Slickers II: The Legend of Curly's Gold
824                              The Spitfire Grill
Name: title, dtype: object

Memory problem bei allen 45k Filmen, als sparse matrix geht es, allerdings geht das umwandeln in np.array dann nicht um damit weiter zu rechnen